# Workbook 4 - Exercises and Answers

Before we start, let's get Colab set up for this workbook:

In [ ]:
%pip install h5py
%pip install lmfit
%pip install numpy
%pip install matplotlib

!git clone https://github.com/timsnow/advanced_sas_training_course
%cd 'advanced_sas_training_course/02 - Data Handling'

With Colab set up, let's set up this workbook for the exercises:

In [ ]:
import h5py
import numpy as np
from lmfit.models import *
import matplotlib.pyplot as plt

hdf_file_path = 'data/i22-363058.h5'
text_file_path = 'data/i22-363110.dat'
internal_hdf_data_path = '/entry/data/data'

Using the `with xxx as yyy:` pattern, load the data contained within the hdf and text files nominated above into the following variables:

 - From the .h5 file load in the `/entry/data/data` dataset as a variable called `two_d_dataset`
 - From the .dat file load in the data, using numpy, and then create both an `x_dataset` and `y_dataset`

In [ ]:
with h5py.File(hdf_file_path, 'r') as file_handle:
    hdf5_dataset = file_handle[internal_hdf_data_path][()]
    
two_d_dataset = hdf5_dataset[0][0]

#########

dat_dataset = np.loadtxt(text_file_path, comments = '#', skiprows = 0, delimiter = '\t')

x_dataset = dat_dataset[:,0]
y_dataset = dat_dataset[:,1]

Using matplotlib create a basic plot of both of these datasets:

In [ ]:
plt.plot(x_dataset, y_dataset)
plt.title('SAXS Data')
plt.yscale('log')
plt.ylabel('Intensity (arb.)')
plt.xscale('log')
plt.xlabel('$q$ (A$^{-1}$)')
plt.plot(x_dataset, y_dataset)
plt.show()

plt.imshow(np.log(two_d_dataset))
plt.show()

Run the following cell to instanciate a function that will let you generate some data:

In [ ]:
def noisy_gaussian_function(x_axis, a, b, c, noise_factor):
    length_of_x_axis = len(x_axis)
    noise_to_add = (np.random.randn(length_of_x_axis) * noise_factor) / (1 / x_axis)
    return (a * np.exp(-((x_axis - b)**2 / (2 * c**2)))) + noise_to_add

Using `numpy` generate a dataset called `x_axis` and then use the function above on that dataset to generate a `y_axis` dataset:

In [ ]:
x_axis = np.linspace(start = 0, stop = 100, num = 1000)
y_axis = noisy_gaussian_function(x_axis, 1, 50, 4, 0.0005)

Use the following cell to plot this data:

In [ ]:
plt.plot(x_axis, y_axis)
plt.show()

Using `lmfit` and the `GaussianModel()` class, fit this data:

In [ ]:
gaussian_model = GaussianModel()
initial_guesses = gaussian_model.guess(data = y_axis, x = x_axis)
gaussian_fit = gaussian_model.fit(data = y_axis, params = initial_guesses, x = x_axis)

In [ ]:
gaussian_fit.plot_fit()
plt.show()
print(gaussian_fit.fit_report())

### Extra credit - for after the training course

Using the `lmfit` webpage on multiple peak fitting, attempt a fit on the following dataset, which is a simulation of an actual SAS dataset:

The following code snippet will be of use:

In [ ]:
def gaussian_function(x_axis, a, b, c):
    return a * np.exp(-((x_axis - b)**2 / (2 * c**2)))

noise_factor = 5000
peak_height = 10
intensity_value = 1
q_data = np.linspace(2, 200, 198)
q_data /= 1000
i_data = intensity_value / q_data**4

peak_one =   gaussian_function(q_data, (i_data[38]  * peak_height), 0.04, 0.001)
peak_two =   gaussian_function(q_data, (i_data[78]  * peak_height), 0.08, 0.001)
peak_three = gaussian_function(q_data, (i_data[118] * peak_height), 0.12, 0.001)
peak_four =  gaussian_function(q_data, (i_data[158] * peak_height), 0.16, 0.001)

i_data += peak_one + peak_two + peak_three + peak_four
di_data = (np.random.rand(len(i_data)) * noise_factor) + np.sqrt(i_data)

plt.fill_between(q_data, i_data - di_data, i_data + di_data, color='gray', alpha=0.5)
plt.yscale('log')
plt.show()

In [ ]:
gauss1 = GaussianModel(prefix='g1_')
gauss2 = GaussianModel(prefix='g2_')
gauss3 = GaussianModel(prefix='g3_')
gauss4 = GaussianModel(prefix='g4_')
background = ExpressionModel('bgval / x**4')
overall_model = gauss1 + gauss2 + gauss3 + gauss4 + background

overall_parameters = gauss1.make_params()
overall_parameters.update(gauss2.make_params())
overall_parameters.update(gauss3.make_params())
overall_parameters.update(gauss4.make_params())
overall_parameters.update(background.make_params())

In [ ]:
for key in overall_parameters:
    if 'amplitude' in key:
        overall_parameters[key].value = 10
        overall_parameters[key].min = 0
        overall_parameters[key].max = 1e10  
    elif 'center' in key:
        overall_parameters[key].min = 0.002
        overall_parameters[key].max = 0.2
    elif 'fwhm' in key:
        overall_parameters[key].value = 0.002
        overall_parameters[key].min = 0
        overall_parameters[key].max = 0.1
    elif 'height' in key:
        overall_parameters[key].value = 20
        overall_parameters[key].min = 0
        overall_parameters[key].max = 1e10      
    elif 'sigma' in key:
        overall_parameters[key].value = 0.005
        overall_parameters[key].min = 0
        overall_parameters[key].max = 0.1

overall_parameters['g1_center'].value = 0.05
overall_parameters['g2_center'].value = 0.07
overall_parameters['g3_center'].value = 0.13
overall_parameters['g4_center'].value = 0.15
overall_parameters['bgval'].value = 1

In [ ]:
overall_fit = overall_model.fit(data = i_data, params = overall_parameters, x = q_data)
overall_fit.plot_fit()
plt.yscale('log')
plt.show()
print(overall_fit.fit_report())